In [5]:
! pip3 install manipulation

  Using cached manipulation-2024.11.7-py3-none-any.whl.metadata (1.9 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached drake-1.34.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (1.5 kB)
  Using cached gradescope_utils-0.5.0-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached mpld3-0.5.10-py3-none-any.whl.metadata (5.1 kB)
  Using cached nevergrad-1.0.5-py3-none-any.whl.metadata (11 kB)
  Using cached stable_baselines3-2.3.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached timeout_decorator-0.5.0-py3-none-any.whl
  Using cached torch-2.3.1-cp312-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached trimesh-4.1.8-py3-none-any.whl.metadata (18 kB)
  Using cached vhacdx-0.0.8.post1-cp312-cp312-macosx_11_0_arm64.whl.metadata (1.1 kB)
  Using cached pydot-3.0.2-py3-none-any.whl.metadata (10 kB)
  Using cached cma-4.0.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached bayesian_optimization-1.4.0-py3-none-any.whl.metadata (469 bytes)
  Using cached c

In [60]:
import os
from copy import deepcopy
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    RandomGenerator,
    ProcessModelDirectives,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    UniformlyRandomRotationMatrix
)
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation, MakeMultibodyPlant
from manipulation.utils import ConfigureParser
from manipulation.systems import AddIiwaDifferentialIK

In [7]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [44]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(
    url="package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf"
)
visualizer.Run()

INFO:drake:Click 'Stop Running' or press Esc to quit


In [10]:
pkg_xml = """<?xml version="1.0"?>
<package format="2">
  <name>bot_shot</name>
  <version>0.0.0</version>
  <description>
    BotShot
  </description>
  <maintainer email="manipulation-student@mit.edu">IIWA</maintainer>
  <author>IIWA</author>
  <license>N/A</license>
</package>
"""

with open("package.xml", "w") as f:
    f.write(pkg_xml)

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().package_map().PopulateFromFolder(os.getcwd())
visualizer.parser().AddModels(url="package://bot_shot/models/table_top.sdf")
visualizer.Run()

INFO:drake:Click 'Stop Running' or press Esc to quit


In [61]:
class CameraSystem:
    def __init__(self, idx, meshcat, diagram, context):
        self.idx = idx

        # Read images
        depth_im_read = (
            diagram.GetOutputPort("camera{}_depth_image".format(idx))
            .Eval(context)
            .data.squeeze()
        )
        self.depth_im = deepcopy(depth_im_read)
        self.depth_im[self.depth_im == np.inf] = 10.0
        self.rgb_im = (
            diagram.GetOutputPort("camera{}_rgb_image".format(idx)).Eval(context).data
        )

        # Visualize
        point_cloud = diagram.GetOutputPort("camera{}_point_cloud".format(idx)).Eval(
            context
        )
        meshcat.SetObject(f"Camera {idx} point cloud", point_cloud)

        # Get other info about the camera
        cam = diagram.GetSubsystemByName("camera" + str(idx))
        cam_context = cam.GetMyMutableContextFromRoot(context)
        self.X_WC = cam.body_pose_in_world_output_port().Eval(cam_context)
        self.cam_info = cam.default_depth_render_camera().core().intrinsics()

In [ ]:
# Note that we are using a different sugar box here, which has a hydroelastic contact model defined
model_directive = """
    directives:
    
    - add_model:
        name: table_top
        file: package://bot_shot/models/table_top.sdf
    - add_weld:
        parent: world
        child: table_top::table_top_center
        X_PC: 
            translation: [0, 0, 0.45]
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/urdf/iiwa14_no_collision.urdf
        default_joint_positions:
            iiwa_joint_1: [1.63]
            iiwa_joint_4: [1.5]
            iiwa_joint_6: [-1.33]

    - add_weld:
        parent: world
        child: iiwa::base
        X_PC:
            translation: [0, 1.2, 0]

    #Add schunk
    - add_model:
        name: wsg
        file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf

    - add_weld:
        parent: iiwa::iiwa_link_7
        child: wsg::body
        X_PC:
            translation: [0, 0, 0.114]
            rotation: !Rpy { deg: [90.0, 0.0, 90.0 ]}
    
    - add_model:
        name: cup
        file: package://bot_shot/models/cup.sdf
        default_free_body_pose:
            textured:
                translation: [0.15,-0.5,0.45]

    - add_model:
        name: ball
        file: package://bot_shot/models/ball.sdf
        default_free_body_pose:
            ball:
                translation: [0,0.65,0.8]

    model_drivers:
        iiwa: !IiwaDriver
            control_mode: position_only
            hand_model_name: wsg
        wsg: !SchunkWsgDriver {}
    plant_config:
        time_step: 1e-3
        #contact_model: "hydroelastic"
        discrete_contact_approximation: "sap"
    """

def preview_scene_given_directives(model_directive):
    meshcat.Delete()
    builder = DiagramBuilder()
    scenario = LoadScenario(data=model_directive)
    station = builder.AddSystem(
        MakeHardwareStation(
            scenario, meshcat, package_xmls=[os.getcwd() + "/package.xml"]
        )
    )
    plant = station.GetSubsystemByName("plant")
    
    # diff_ik_plant = MakeMultibodyPlant(scenario, model_instance_names=["iiwa"])
    # differential_ik = AddIiwaDifferentialIK(builder, diff_ik_plant)
    # builder.Connect(
    #     differential_ik.get_output_port(),
    #     station.GetInputPort("iiwa.position"),
    # )
    # builder.Connect(
    #     station.GetOutputPort("iiwa.state_estimated"),
    #     differential_ik.GetInputPort("robot_state"),
    # )

    iiwa_position = builder.AddSystem(ConstantVectorSource([1.63, 0.0, 0.0, 1.2, 0.0, -1.6, 0.0]))
    builder.Connect(
        iiwa_position.get_output_port(),
        station.GetInputPort("iiwa.position"),
    )

    wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
    builder.Connect(
        wsg_position.get_output_port(), station.GetInputPort("wsg.position")
    )
   
    diagram = builder.Build()

    context = plant.CreateDefaultContext()
    gripper = plant.GetBodyByName("body")
    table = plant.GetBodyByName("table_top_link")

    initial_pose = plant.EvalBodyPoseInWorld(context, gripper)
    # X_T = plant.EvalBodyPoseInWorld(context, table)
    # rng = np.random.default_rng()
    # for body_index in plant.GetFloatingBaseBodies():
    #     generator = RandomGenerator(rng.integers(1000))  # this is for c++
    #     tf = RigidTransform(
    #         UniformlyRandomRotationMatrix(generator),
    #         [rng.uniform(-0.15, 0.15), rng.uniform(-0.2, 0.2), 0],
    #     )
    #     plant.SetFreeBodyPose(
    #         context, plant.get_body(body_index), X_T.multiply(tf)
    #     )

    cameras = []
    cameras.append(CameraSystem(0, meshcat, diagram, context))
    cameras.append(CameraSystem(1, meshcat, diagram, context))

    simulator = Simulator(diagram)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()

preview_scene_given_directives(model_directive)

RuntimeError: System _ does not have an output port named camera0_depth_image (valid port names: it has no output ports)

: 